In [14]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, SMOL_LM_135M, GRPO_MATH_CONFIG

cfg = GRPO_MATH_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module()

2025-03-22 14:22:50.564 | INFO     | trl_wrapper.trainer_wrapper:init_model:256 - Using device: mps
2025-03-22 14:22:50.565 | INFO     | trl_wrapper.trainer_wrapper:init_model:270 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa
2025-03-22 14:22:51.242 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:296 - Using chat template override: smollmv2
2025-03-22 14:22:51.243 | INFO     | trl_wrapper.wrapper_config:__init__:126 - Cache dir: ../dataset_caches/g_s_m8_k_data_module
Map: 100%|██████████| 7473/7473 [00:00<00:00, 54413.99 examples/s]


In [16]:
wrapper.init_trainer()

2025-03-22 14:22:53.215 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:333 - Saving output to: ./runs/03-22-14-22-882269-smollm2-135m-instruct-
2025-03-22 14:22:53.217 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:356 - Initializing trainer, run_name: 03-22-14-22-882269-smollm2-135m-instruct-, wandb project: qwen-math-grpo
2025-03-22 14:22:53.217 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:441 - Using vllm: False


In [ ]:
wrapper.train()

2025-03-22 14:22:54.716 | INFO     | trl_wrapper.trainer_wrapper:train:609 - Starting training.
